<a href="https://colab.research.google.com/github/NicolasFerre/ColabFerre/blob/dev/Consolidaci%C3%B3n_de_pedidos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
#!pip install xlsxwriter
#!pip install python-calamine
#!pip install fastexcel

In [41]:
# Probando polars, no pregunten xd
import polars as ps
import pandas as pd
import numpy as np

In [42]:
reportes_cantidad = input("Cantidad de reportes: ")
reportes_cantidad = int(reportes_cantidad)

Cantidad de reportes: 3


In [43]:
transacciones_cantidad = input("Cantidad de transacciones: ")
transacciones_cantidad = int(transacciones_cantidad)

Cantidad de transacciones: 57


In [44]:
archivos = []
for i in range(reportes_cantidad):
    archivos.append((f"Reportes/Reporte{i+1}.txt"))

In [45]:
reportes = ps.read_csv(archivos[0], separator="\t")

In [46]:
if len(archivos) > 1:
  for i in range(1, reportes_cantidad):
      reportes = reportes.vstack(ps.read_csv(archivos[i], separator="\t", schema=reportes.schema, truncate_ragged_lines=True))
  print(f"Uniendo {reportes_cantidad} reportes")


Uniendo 3 reportes


In [47]:
reportes.write_csv("Reporte.txt", separator="\t")

In [48]:
transacciones = ps.read_csv("Transacciones/Transacciones1.csv", separator = ',')

In [49]:
archivos = []
for i in range(transacciones_cantidad):
    archivos.append((f"Transacciones/Transacciones{i+1}.csv"))

In [50]:
if len(archivos) > 1:
  for i in range(1, transacciones_cantidad):
      transacciones = transacciones.vstack(ps.read_csv(archivos[i], separator = ',', schema=transacciones.schema, truncate_ragged_lines=True))

In [51]:
transacciones.write_csv("Transacciones.csv", separator=",")

In [52]:
transacciones = transacciones.rename({'Número de pedido':'amazon-order-id'})

In [53]:
df = reportes.join(transacciones, on='amazon-order-id', how='left', suffix='_transacciones')

In [54]:
formato_num = {ps.Int64:"0", ps.Float64:"0.00"}

In [55]:
df.write_excel("ConsolidacionReportes.xlsx", worksheet='Reporte', dtype_formats=formato_num)

In [56]:
df = transacciones.join(reportes, on='amazon-order-id', how='left', suffix='_reportes')

In [57]:
df.write_excel("ConsolidacionTransacciones.xlsx", worksheet='Transacciones', dtype_formats=formato_num)